In [4]:
import numpy as np
import pandas as pd

In [5]:
toxicWordsTrain=pd.read_csv("dataset/jigsaw-toxic-comment-classification-challenge/train.csv").fillna(' ')
toxicWordsTest=pd.read_csv("dataset/jigsaw-toxic-comment-classification-challenge/test.csv").fillna(' ')

#Class labels
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

y_train = toxicWordsTrain[list_classes].values
x_train = toxicWordsTrain["comment_text"]
x_test  = toxicWordsTest["comment_text"]

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Create tokenizer
tokenizer = Tokenizer(num_words=None,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True,
                      split=" ",
                      char_level=False)

# Fit and run tokenizer
tokenizer.fit_on_texts(list(x_train))
tokenized_train = tokenizer.texts_to_sequences(x_train)
tokenized_test = tokenizer.texts_to_sequences(x_test)
word_index = tokenizer.word_index


# Extract variables
vocab_size = len(word_index)
longest = max(len(seq) for seq in tokenized_train)
average = np.mean([len(seq) for seq in tokenized_train])
stdev = np.std([len(seq) for seq in tokenized_train])
max_len = int(average + stdev * 3)


In [7]:
# Pad sequences
processed_X_train = pad_sequences(tokenized_train, maxlen=max_len, padding='post', truncating='post')
processed_X_test = pad_sequences(tokenized_test, maxlen=max_len, padding='post', truncating='post')

In [8]:
print(processed_X_train.shape)
print(y_train.shape)

(159571, 371)
(159571, 6)


In [9]:
import tensorflow as tf
model = tf.keras.models.load_model("model.best.hdf5")

In [10]:

import matplotlib.pyplot as plt

def create_bar_chart(prediction):
    # Prepare data for bar chart
    labels = ['Toxic', 'Severe Toxic', 'Obscene', 'Threat', 'Insult', 'Identity Hate']
    values = prediction[0]  # Assuming prediction returns probabilities for each category
    
    colors = ['skyblue', 'lightcoral', 'yellowgreen', 'gold', 'violet', 'orange']
    # Create bar chart
    plt.figure(figsize=(10, 5))
    bars = plt.bar(labels, values, color=colors)
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, f'{yval * 100:.0f}%', ha='center', va='bottom')
    plt.yticks(np.arange(0, 1.1, 0.1), [f'{int(i * 100)}%' for i in np.arange(0, 1.1, 0.1)])
    plt.ylabel('Probability')
    plt.title('Toxicity Levels')
    
    # Save the plot to a file
    plt.savefig('toxicity_levels.png')
    plt.close()

    return 'toxicity_levels.png'

In [11]:

def toxicity_level(string):
    """
    Return toxicity probability based on inputed string.
    """
    # Process string
    new_string = [string]
    new_string = tokenizer.texts_to_sequences(new_string)
    new_string = pad_sequences(new_string, maxlen=max_len, padding='post', truncating='post')
    
    # Predict
    prediction = model.predict(new_string)
    
    # Print output
    response = f"Toxicity levels for '{string}': \n" \
                f"Toxic:         {prediction[0][0]:.0%} \n" \
                f"Severe Toxic:  {prediction[0][1]:.0%} \n" \
                f"Obscene:       {prediction[0][2]:.0%} \n" \
                f"Threat:        {prediction[0][3]:.0%} \n" \
                f"Insult:        {prediction[0][4]:.0%} \n" \
                f"Identity Hate: {prediction[0][5]:.0%} \n"
    
    bar_chart_file = create_bar_chart(prediction)

    return response, bar_chart_file

In [ ]:
import google.generativeai as genai

GOOGLE_API_KEY = ""
genai.configure(api_key=GOOGLE_API_KEY)
geminibot = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
def statistic_evaluate(stat):
    question = 'Comment on this statistic: ' + stat
    response = geminibot.generate_content(question)
    response.resolve()
    
    return [{"role": "assistant", "content": response.text}]

In [14]:
import gradio as gr

with gr.Blocks() as demo:

    textbox = gr.Textbox(label="Toxic Comment Detection for:", lines=2, placeholder="Enter your comment here...")
    statistic_output = gr.Textbox(label="Statistic", lines=3)
    bar_chart = gr.Image(label="Bar Chart", type="filepath")

    interface = gr.Interface(
    fn = toxicity_level,
    inputs = textbox, #gr.Textbox(label="Toxic Comment Detection for:", lines=2, placeholder="Enter your comment here..."),
    outputs = [statistic_output, bar_chart], #[gr.Textbox(label="Statistic", lines=3), gr.Image(label="Bar Chart", type="filepath")],
    title = "Toxic Comment Detection",
    description = "Enter a comment to check if it's toxic or not."
    )

    chatbot = gr.Chatbot(
        type = "messages"
    )
    button = gr.Button("Evaluate the result")
    button.click(statistic_evaluate, inputs=statistic_output, outputs=chatbot)

# Launch the interface
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://2c0eefb425b9311143.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
